# Firstly playin with the train set :)

In [2]:
!pip install textblob

In [3]:
import numpy as np
import pandas as pd

from sklearn.metrics import confusion_matrix, classification_report

In [4]:
df = pd.read_csv(r'Desktop\Train_Data.csv')
df_test = pd.read_csv(r'Desktop\Test_Data.csv')

df.head()

,headline,is_sarcastic
0,supreme court votes 7-2 to legalize all worldl...,1
1,hungover man horrified to learn he made dozens...,1
2,emily's list founder: women are the 'problem s...,0
3,send your kids back to school with confidence,0
4,watch: experts talk pesticides and health,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44262 entries, 0 to 44261
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   headline      44262 non-null  object
 1   is_sarcastic  44262 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 691.7+ KB


In [6]:
df.isna().sum()

headline        0
is_sarcastic    0
dtype: int64

In [7]:
df.value_counts()

headline                                                                                       is_sarcastic
the 20 funniest tweets from women this week                                                    0               16
sunday roundup                                                                                 0               13
no way to prevent this,' says only nation where this regularly happens                         1               12
the funniest tweets from parents this week                                                     0               10
the funniest tweets from women this week                                                       0                5
                                                                                                               ..
cia finds definitive evidence of second shooter in jfk assassination                           1                1
cia chief warns of 'tremendous' consequences for iran                                         

In [8]:
df['is_sarcastic'].value_counts()

0    23958
1    20304
Name: is_sarcastic, dtype: int64

In [9]:
df[df['headline'] == ''].sum()

headline        0.0
is_sarcastic    0.0
dtype: float64

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[['headline']], df['is_sarcastic'], random_state=42)
X_train.shape, X_test.shape

((33196, 1), (11066, 1))

In [11]:
from collections import Counter
Counter(y_train), Counter(y_test)

(Counter({1: 15185, 0: 18011}), Counter({1: 5119, 0: 5947}))

In [12]:
import string

X_train['char_count'] = X_train['headline'].apply(len)
X_train['word_count'] = X_train['headline'].apply(lambda x: len(x.split()))
X_train['word_density'] = X_train['char_count'] / (X_train['word_count']+1)
X_train['punctuation_count'] = X_train['headline'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
X_train['title_word_count'] = X_train['headline'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
X_train['upper_case_word_count'] = X_train['headline'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))


X_test['char_count'] = X_test['headline'].apply(len)
X_test['word_count'] = X_test['headline'].apply(lambda x: len(x.split()))
X_test['word_density'] = X_test['char_count'] / (X_test['word_count']+1)
X_test['punctuation_count'] = X_test['headline'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
X_test['title_word_count'] = X_test['headline'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
X_test['upper_case_word_count'] = X_test['headline'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

In [13]:
X_train.head()

,headline,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count
34059,authoritarian secretary of transportation decl...,106,15,6.625000,0,0,0
6098,study: marriages between perfectly matched cou...,88,13,6.285714,1,0,0
6252,employee wellness programs aren't so voluntary...,54,7,6.750000,1,0,0
20826,p is for p*ssy' is the alphabet book of your w...,55,12,4.230769,2,0,0
24719,nothing going right for area surgeon today,42,7,5.250000,0,0,0


In [14]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=1, random_state=42, solver='liblinear')

In [15]:
lr.fit(X_train.drop(['headline'], axis=1), y_train)
predictions = lr.predict(X_test.drop(['headline'], axis=1))

print(classification_report(y_test, predictions))
pd.DataFrame(confusion_matrix(y_test, predictions))

              precision    recall  f1-score   support

           0       0.62      0.72      0.66      5947
           1       0.60      0.48      0.53      5119

    accuracy                           0.61     11066
   macro avg       0.61      0.60      0.60     11066
weighted avg       0.61      0.61      0.60     11066



,0,1
0,4287,1660
1,2679,2440


In [16]:
import textblob
x_train_snt_obj = X_train['headline'].apply(lambda row: textblob.TextBlob(row).sentiment)
X_train['Polarity'] = [obj.polarity for obj in x_train_snt_obj.values]
X_train['Subjectivity'] = [obj.subjectivity for obj in x_train_snt_obj.values]

x_test_snt_obj = X_test['headline'].apply(lambda row: textblob.TextBlob(row).sentiment)
X_test['Polarity'] = [obj.polarity for obj in x_test_snt_obj.values]
X_test['Subjectivity'] = [obj.subjectivity for obj in x_test_snt_obj.values]

In [17]:
X_train.head()

,headline,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,Polarity,Subjectivity
34059,authoritarian secretary of transportation decl...,106,15,6.625000,0,0,0,0.142857,0.767857
6098,study: marriages between perfectly matched cou...,88,13,6.285714,1,0,0,0.333333,0.688889
6252,employee wellness programs aren't so voluntary...,54,7,6.750000,1,0,0,0.000000,0.000000
20826,p is for p*ssy' is the alphabet book of your w...,55,12,4.230769,2,0,0,-0.100000,0.400000
24719,nothing going right for area surgeon today,42,7,5.250000,0,0,0,0.285714,0.535714


In [18]:
lr.fit(X_train.drop(['headline'], axis=1), y_train, )
predictions = lr.predict(X_test.drop(['headline'], axis=1))

print(classification_report(y_test, predictions))
pd.DataFrame(confusion_matrix(y_test, predictions))

              precision    recall  f1-score   support

           0       0.62      0.72      0.67      5947
           1       0.60      0.49      0.54      5119

    accuracy                           0.61     11066
   macro avg       0.61      0.60      0.60     11066
weighted avg       0.61      0.61      0.61     11066



,0,1
0,4293,1654
1,2624,2495


In [19]:
!pip install contractions
!pip install textsearch
!pip install tqdm
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\berke\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\berke\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
import contractions
contractions.fix('I didn\'t like this t-shirt')

'I did not like this t-shirt'

In [21]:
import nltk
import contractions
import re

# remove some stopwords to capture negation in n-grams if possible
stop_words = nltk.corpus.stopwords.words('english')
stop_words.remove('no')
stop_words.remove('not')
stop_words.remove('but')

# load up a simple porter stemmer - nothing fancy
ps = nltk.porter.PorterStemmer()

def simple_text_preprocessor(document): 
    # lower case
    document = str(document).lower()
    
    # expand contractions
    document = contractions.fix(document)
    
    # remove unnecessary characters
    document = re.sub(r'[^a-zA-Z]',r' ', document)
    document = re.sub(r'nbsp', r'', document)
    document = re.sub(' +', ' ', document)
    
    # simple porter stemming
    document = ' '.join([ps.stem(word) for word in document.split()])
    
    # stopwords removal
    document = ' '.join([word for word in document.split() if word not in stop_words])
    
    return document

stp = np.vectorize(simple_text_preprocessor)
X_train['Clean Headline'] = stp(X_train['headline'].values)
X_test['Clean Headline'] = stp(X_test['headline'].values)

X_train.head()

,headline,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,Polarity,Subjectivity,Clean Headline
34059,authoritarian secretary of transportation decl...,106,15,6.625000,0,0,0,0.142857,0.767857,authoritarian secretari transport declar ha ul...
6098,study: marriages between perfectly matched cou...,88,13,6.285714,1,0,0,0.333333,0.688889,studi marriag perfectli match coupl still onli...
6252,employee wellness programs aren't so voluntary...,54,7,6.750000,1,0,0,0.000000,0.000000,employe well program not voluntari anymor
20826,p is for p*ssy' is the alphabet book of your w...,55,12,4.230769,2,0,0,-0.100000,0.400000,p p ssi alphabet book wet dream
24719,nothing going right for area surgeon today,42,7,5.250000,0,0,0,0.285714,0.535714,noth go right area surgeon today


In [22]:
X_train_metadata = X_train.drop(['headline', 'Clean Headline'], axis=1).reset_index(drop=True)
X_test_metadata = X_test.drop(['headline', 'Clean Headline'], axis=1).reset_index(drop=True)

X_train_metadata.head()

,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,Polarity,Subjectivity
0,106,15,6.625000,0,0,0,0.142857,0.767857
1,88,13,6.285714,1,0,0,0.333333,0.688889
2,54,7,6.750000,1,0,0,0.000000,0.000000
3,55,12,4.230769,2,0,0,-0.100000,0.400000
4,42,7,5.250000,0,0,0,0.285714,0.535714


In [23]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df=0.0, max_df=1.0, ngram_range=(1, 1))
X_traincv = cv.fit_transform(X_train['Clean Headline']).toarray()
X_traincv = pd.DataFrame(X_traincv, columns=cv.get_feature_names())

X_testcv = cv.transform(X_test['Clean Headline']).toarray()
X_testcv = pd.DataFrame(X_testcv, columns=cv.get_feature_names())
X_traincv.head()

C:\Users\berke\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\berke\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aaa,aaron,aarp,ab,abandon,abaya,abba,abbey,abbi,abc,...,zoo,zookeep,zooland,zoologist,zoom,zoroastrian,zsa,zucker,zuckerberg,zz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
X_train_comb = pd.concat([X_train_metadata, X_traincv], axis=1)
X_test_comb = pd.concat([X_test_metadata, X_testcv], axis=1)

X_train_comb.head()

,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,Polarity,Subjectivity,aaa,aaron,...,zoo,zookeep,zooland,zoologist,zoom,zoroastrian,zsa,zucker,zuckerberg,zz
0,106,15,6.625000,0,0,0,0.142857,0.767857,0,0,...,0,0,0,0,0,0,0,0,0,0
1,88,13,6.285714,1,0,0,0.333333,0.688889,0,0,...,0,0,0,0,0,0,0,0,0,0
2,54,7,6.750000,1,0,0,0.000000,0.000000,0,0,...,0,0,0,0,0,0,0,0,0,0
3,55,12,4.230769,2,0,0,-0.100000,0.400000,0,0,...,0,0,0,0,0,0,0,0,0,0
4,42,7,5.250000,0,0,0,0.285714,0.535714,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
lr.fit(X_train_comb, y_train)
predictions = lr.predict(X_test_comb)

print(classification_report(y_test, predictions))
pd.DataFrame(confusion_matrix(y_test, predictions))

              precision    recall  f1-score   support

           0       0.87      0.90      0.88      5947
           1       0.88      0.84      0.86      5119

    accuracy                           0.87     11066
   macro avg       0.87      0.87      0.87     11066
weighted avg       0.87      0.87      0.87     11066



,0,1
0,5358,589
1,823,4296


In [26]:
df_test['char_count'] = df_test['headline'].apply(len)
df_test['word_count'] = df_test['headline'].apply(lambda x: len(x.split()))
df_test['word_density'] = df_test['char_count'] / (df_test['word_count']+1)
df_test['punctuation_count'] = df_test['headline'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
df_test['title_word_count'] = df_test['headline'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
df_test['upper_case_word_count'] = df_test['headline'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

df_test.head()

,headline,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count
0,area stand-up comedian questions the deal with...,65,9,6.500000,2,0,0
1,dozens of glowing exit signs mercilessly taunt...,65,9,6.500000,0,0,0
2,perfect response to heckler somewhere in prop ...,62,9,6.200000,1,0,0
3,gop prays for ossoff lossoff,28,5,4.666667,0,0,0
4,trevor noah says the scary truth about trump's...,65,11,5.416667,1,0,0


In [27]:
df_test.isna().sum()

headline                 0
char_count               0
word_count               0
word_density             0
punctuation_count        0
title_word_count         0
upper_case_word_count    0
dtype: int64

In [28]:
df_test_snt_obj = df_test['headline'].apply(lambda row: textblob.TextBlob(row).sentiment)
df_test['Polarity'] = [obj.polarity for obj in df_test_snt_obj.values]
df_test['Subjectivity'] = [obj.subjectivity for obj in df_test_snt_obj.values]

In [29]:
df_test['Clean Headline'] = stp(df_test['headline'].values)

df_test.head()

,headline,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,Polarity,Subjectivity,Clean Headline
0,area stand-up comedian questions the deal with...,65,9,6.500000,2,0,0,0.0,0.0,area stand comedian question deal drive thru w...
1,dozens of glowing exit signs mercilessly taunt...,65,9,6.500000,0,0,0,-0.7,1.0,dozen glow exit sign mercilessli taunt multipl...
2,perfect response to heckler somewhere in prop ...,62,9,6.200000,1,0,0,1.0,1.0,perfect respons heckler somewher prop comedian...
3,gop prays for ossoff lossoff,28,5,4.666667,0,0,0,0.0,0.0,gop pray ossoff lossoff
4,trevor noah says the scary truth about trump's...,65,11,5.416667,1,0,0,0.0,0.8,trevor noah say scari truth trump rumor love c...


In [30]:
df_test_metadata = df_test.drop(['headline', 'Clean Headline'], axis=1).reset_index(drop=True)

In [31]:
df_test_metadata.head()

,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,Polarity,Subjectivity
0,65,9,6.500000,2,0,0,0.0,0.0
1,65,9,6.500000,0,0,0,-0.7,1.0
2,62,9,6.200000,1,0,0,1.0,1.0
3,28,5,4.666667,0,0,0,0.0,0.0
4,65,11,5.416667,1,0,0,0.0,0.8


In [32]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df=0.0, max_df=1.0, ngram_range=(1, 1))
X_traincv = cv.fit_transform(X_train['Clean Headline']).toarray()
X_traincv = pd.DataFrame(X_traincv, columns=cv.get_feature_names())

df_testcv = cv.transform(df_test['Clean Headline']).toarray()
df_testcv = pd.DataFrame(df_testcv, columns=cv.get_feature_names())
df_testcv.head()

C:\Users\berke\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\berke\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aaa,aaron,aarp,ab,abandon,abaya,abba,abbey,abbi,abc,...,zoo,zookeep,zooland,zoologist,zoom,zoroastrian,zsa,zucker,zuckerberg,zz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
df_test_comb = pd.concat([df_test_metadata, df_testcv], axis=1)

In [34]:
df_test_comb.head()

,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,Polarity,Subjectivity,aaa,aaron,...,zoo,zookeep,zooland,zoologist,zoom,zoroastrian,zsa,zucker,zuckerberg,zz
0,65,9,6.500000,2,0,0,0.0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,65,9,6.500000,0,0,0,-0.7,1.0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,9,6.200000,1,0,0,1.0,1.0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,5,4.666667,0,0,0,0.0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,65,11,5.416667,1,0,0,0.0,0.8,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
prediction = pd.DataFrame()

lr.fit(X_train_comb, y_train)
prediction["prediction"] = lr.predict(df_test_comb)
prediction.head(20)

,prediction
0,1
1,1
2,1
3,0
4,0
5,0
6,1
7,1
8,0
9,0


# Here comes the actual job!!

In [36]:
X_train = pd.DataFrame()
y_train = pd.DataFrame()
X_train['headline'] = df["headline"]
y_train["is_sarcastic"] = df["is_sarcastic"]

In [37]:
import string

X_train['char_count'] = X_train['headline'].apply(len)
X_train['word_count'] = X_train['headline'].apply(lambda x: len(x.split()))
X_train['word_density'] = X_train['char_count'] / (X_train['word_count']+1)
X_train['punctuation_count'] = X_train['headline'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
X_train['title_word_count'] = X_train['headline'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
X_train['upper_case_word_count'] = X_train['headline'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))


df_test['char_count'] = df_test['headline'].apply(len)
df_test['word_count'] = df_test['headline'].apply(lambda x: len(x.split()))
df_test['word_density'] = df_test['char_count'] / (df_test['word_count']+1)
df_test['punctuation_count'] = df_test['headline'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
df_test['title_word_count'] = df_test['headline'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
df_test['upper_case_word_count'] = df_test['headline'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

In [38]:
import textblob
x_train_snt_obj = X_train['headline'].apply(lambda row: textblob.TextBlob(row).sentiment)
X_train['Polarity'] = [obj.polarity for obj in x_train_snt_obj.values]
X_train['Subjectivity'] = [obj.subjectivity for obj in x_train_snt_obj.values]

df_test_snt_obj = df_test['headline'].apply(lambda row: textblob.TextBlob(row).sentiment)
df_test['Polarity'] = [obj.polarity for obj in df_test_snt_obj.values]
df_test['Subjectivity'] = [obj.subjectivity for obj in df_test_snt_obj.values]

In [39]:
!pip install contractions
!pip install textsearch
!pip install tqdm
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\berke\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\berke\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [40]:
import nltk
import contractions
import re

# remove some stopwords to capture negation in n-grams if possible
stop_words = nltk.corpus.stopwords.words('english')
stop_words.remove('no')
stop_words.remove('not')
stop_words.remove('but')

# load up a simple porter stemmer - nothing fancy
ps = nltk.porter.PorterStemmer()

def simple_text_preprocessor(document): 
    # lower case
    document = str(document).lower()
    
    # expand contractions
    document = contractions.fix(document)
    
    # remove unnecessary characters
    document = re.sub(r'[^a-zA-Z]',r' ', document)
    document = re.sub(r'nbsp', r'', document)
    document = re.sub(' +', ' ', document)
    
    # simple porter stemming
    document = ' '.join([ps.stem(word) for word in document.split()])
    
    # stopwords removal
    document = ' '.join([word for word in document.split() if word not in stop_words])
    
    return document

stp = np.vectorize(simple_text_preprocessor)
X_train['Clean Headline'] = stp(X_train['headline'].values)
df_test['Clean Headline'] = stp(df_test['headline'].values)

X_train.head()

,headline,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,Polarity,Subjectivity,Clean Headline
0,supreme court votes 7-2 to legalize all worldl...,53,9,5.300000,1,0,0,0.0,0.000000,suprem court vote legal worldli vice
1,hungover man horrified to learn he made dozens...,66,12,5.076923,0,0,0,0.0,0.066667,hungov man horrifi learn made dozen plan last ...
2,emily's list founder: women are the 'problem s...,65,10,5.909091,4,0,0,0.0,0.000000,emili list founder women problem solver congress
3,send your kids back to school with confidence,45,8,5.000000,0,0,0,0.0,0.000000,send kid back school confid
4,watch: experts talk pesticides and health,41,6,5.857143,1,0,0,0.0,0.000000,watch expert talk pesticid health


In [41]:
X_train_metadata = X_train.drop(['headline', 'Clean Headline'], axis=1).reset_index(drop=True)
df_test_metadata = df_test.drop(['headline', 'Clean Headline'], axis=1).reset_index(drop=True)

X_train_metadata.head()

,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,Polarity,Subjectivity
0,53,9,5.300000,1,0,0,0.0,0.000000
1,66,12,5.076923,0,0,0,0.0,0.066667
2,65,10,5.909091,4,0,0,0.0,0.000000
3,45,8,5.000000,0,0,0,0.0,0.000000
4,41,6,5.857143,1,0,0,0.0,0.000000


In [42]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df=0.0, max_df=1.0, ngram_range=(1, 1))
X_traincv = cv.fit_transform(X_train['Clean Headline']).toarray()
X_traincv = pd.DataFrame(X_traincv, columns=cv.get_feature_names())

df_testcv = cv.transform(df_test['Clean Headline']).toarray()
df_testcv = pd.DataFrame(df_testcv, columns=cv.get_feature_names())
df_testcv

C:\Users\berke\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\berke\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aaa,aaron,aarp,aatish,ab,abandon,abaya,abba,abbey,abbi,...,zoo,zookeep,zooland,zoologist,zoom,zoroastrian,zsa,zucker,zuckerberg,zz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11061,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11062,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11063,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11064,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
X_train_comb = pd.concat([X_train_metadata, X_traincv], axis=1)
df_test_comb = pd.concat([df_test_metadata, df_testcv], axis=1)

X_train_comb.head()

,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,Polarity,Subjectivity,aaa,aaron,...,zoo,zookeep,zooland,zoologist,zoom,zoroastrian,zsa,zucker,zuckerberg,zz
0,53,9,5.300000,1,0,0,0.0,0.000000,0,0,...,0,0,0,0,0,0,0,0,0,0
1,66,12,5.076923,0,0,0,0.0,0.066667,0,0,...,0,0,0,0,0,0,0,0,0,0
2,65,10,5.909091,4,0,0,0.0,0.000000,0,0,...,0,0,0,0,0,0,0,0,0,0
3,45,8,5.000000,0,0,0,0.0,0.000000,0,0,...,0,0,0,0,0,0,0,0,0,0
4,41,6,5.857143,1,0,0,0.0,0.000000,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=1, random_state=42, solver='liblinear')

In [45]:
predictions = pd.DataFrame()
lr.fit(X_train_comb.iloc[0:35000], y_train.iloc[0:35000])
predictions["prediction"] = lr.predict(df_test_comb)

predictions.value_counts()

C:\Users\berke\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


prediction
0             6291
1             4775
dtype: int64

In [46]:
predictions.head(20)

,prediction
0,1
1,1
2,1
3,0
4,0
5,0
6,1
7,1
8,0
9,0


In [47]:
predictions.to_csv('Desktop\submission.csv', index=False)